# Notebook for answering questions from email

In [41]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from acquire import get_cohort_data
from prepare import split_by_program
from wrangle import wrangle_cohort_data

from sklearn import metrics
from env import user, host, password

import warnings
warnings.filterwarnings("ignore")

In [2]:
# getting the cohort data
df = get_cohort_data()

In [3]:
df.head() # check_yo_head

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0


In [4]:
# looking at the shape of the data
df.shape

(847330, 15)

In [5]:
# checking for nulls in the data
df.isna().sum()

date               0
time               0
path               1
user_id            0
cohort_id          0
ip                 0
id            774925
name          774925
slack         774925
start_date    774925
end_date      774925
created_at    774925
updated_at    774925
deleted_at    847330
program_id    774925
dtype: int64

In [6]:
# reading the info_cohorts.csv that was attached to the email
info_df = pd.read_csv("info_cohorts.csv",index_col = 0)

In [7]:
info_df.head() # check_yo_head

,name,start_date,end_date,program_id
cohort_id,,,,
1,Arches,2014-02-04,2014-04-22,1
2,Badlands,2014-06-04,2014-08-22,1
3,Carlsbad,2014-09-04,2014-11-05,1
4,Denali,2014-10-20,2015-01-18,1
5,Everglades,2014-11-18,2015-02-24,1


In [8]:
# taking a look at the whole .csv
info_df

,name,start_date,end_date,program_id
cohort_id,,,,
1,Arches,2014-02-04,2014-04-22,1
2,Badlands,2014-06-04,2014-08-22,1
3,Carlsbad,2014-09-04,2014-11-05,1
4,Denali,2014-10-20,2015-01-18,1
5,Everglades,2014-11-18,2015-02-24,1
6,Franklin,2015-02-03,2015-05-26,1
7,Glacier,2015-06-05,2015-10-06,1
8,Hampton,2015-09-22,2016-02-06,1
9,Apollo,2015-03-30,2015-07-29,4


In [9]:
# changing cohort_id to an integer
df = df.astype({"cohort_id": int})

In [10]:
df.head() # check_yo_head

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0


In [11]:
# joining the two datframes I have
join_df = df.join(info_df,on = 'cohort_id',how = 'outer',lsuffix = 'str')

In [12]:
join_df.head() # check_yo_head

,date,time,path,user_id,cohort_id,ip,id,namestr,slack,start_datestr,end_datestr,created_at,updated_at,deleted_at,program_idstr,name,start_date,end_date,program_id
0.0,2018-01-26,09:55:03,/,1.0,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,Hampton,2015-09-22,2016-02-06,1.0
1.0,2018-01-26,09:56:02,java-ii,1.0,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,Hampton,2015-09-22,2016-02-06,1.0
2.0,2018-01-26,09:56:05,java-ii/object-oriented-programming,1.0,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,Hampton,2015-09-22,2016-02-06,1.0
3.0,2018-01-26,09:56:06,slides/object_oriented_programming,1.0,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,Hampton,2015-09-22,2016-02-06,1.0
58.0,2018-01-26,10:40:15,javascript-i/functions,1.0,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,Hampton,2015-09-22,2016-02-06,1.0


In [13]:
# looking at null values in the data
join_df.isna().sum()

date                  5
time                  5
path                  6
user_id               5
cohort_id             0
ip                    5
id               774930
namestr          774930
slack            774930
start_datestr    774930
end_datestr      774930
created_at       774930
updated_at       774930
deleted_at       847335
program_idstr    774930
name              65245
start_date        65245
end_date          65245
program_id        65245
dtype: int64

In [14]:
# using my wranlge_cohort_data function to do some clean up
df = wrangle_cohort_data()

In [15]:
df.head() # check_yo_head

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,count_helper,program_name,program_subdomain,slack,module/lesson
datetime,,,,,,,,,,,,,
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,NaN
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,java-ii/object-oriented-programming
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,slides/object_oriented_programming
2018-01-26 10:40:15,javascript-i/functions,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,javascript-i/functions
2018-01-26 11:26:13,java-i,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,NaN


In [16]:
# taking a look at the shape of the data
df.shape

(739722, 13)

In [18]:
# using .info() to look at Dtype
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 739722 entries, 2018-01-26 09:56:02 to 2021-04-21 16:32:36
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   path               739722 non-null  object 
 1   user_id            739722 non-null  int64  
 2   cohort_id          739722 non-null  int64  
 3   ip                 739722 non-null  object 
 4   cohort_name        739722 non-null  object 
 5   start_date         739722 non-null  object 
 6   end_date           739722 non-null  object 
 7   program_id         739722 non-null  float64
 8   count_helper       739722 non-null  int64  
 9   program_name       739722 non-null  object 
 10  program_subdomain  739722 non-null  object 
 11  slack              739722 non-null  object 
 12  module/lesson      602162 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 79.0+ MB


In [32]:
# trying a groupby 'path'
df.groupby('path').sum().head()

,user_id,cohort_id,program_id,count_helper
path,,,,
%20https://github.com/RaulCPena,580,55,3.0,1
',64,28,2.0,1
",%20https://github.com/RaulCPena",580,55,3.0,1
.git,203,24,2.0,1
.gitignore,899,83,5.0,2


In [33]:
# trying a groupby 'program_name'
df.groupby('program_name').count_helper.sum()

program_name
Data Science                        73475
Front End Web Development               4
Java Full Stack Web Development    637376
PHP Full Stack Web Development      28867
Name: count_helper, dtype: int64

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [38]:
# looking at all the programs
df.program_name.unique()

array(['PHP Full Stack Web Development',
       'Java Full Stack Web Development', 'Front End Web Development',
       'Data Science'], dtype=object)

In [44]:
# using prepare function to split into programs
php_df,java_df,ds_df,fe_df = split_by_program(df)

In [42]:
#php_df
php_df.groupby('cohort_name')['path','user_id'].nunique()

,path,user_id
cohort_name,,
Arches,341,18
Badlands,43,3
Denali,3,1
Franklin,44,2
Glacier,142,6
Hampton,277,5
Ike,94,5
Joshua,116,5
Kings,347,9


In [47]:
# idxmax to get the most viewed module/lesson
php_df.groupby(['module/lesson','cohort_name']).path.count().idxmax()

('spring/fundamentals', 'Lassen')

In [43]:
# java_df
java_df.groupby('cohort_name')['path','user_id'].nunique()

,path,user_id
cohort_name,,
Andromeda,365,28
Apex,272,24
Bash,274,23
Betelgeuse,333,21
Ceres,300,29
Deimos,287,27
Europa,266,28
Fortuna,257,29
Ganymede,295,28


In [51]:
# idxmax to get the most viewed module/lesson
java_df.groupby(['module/lesson','cohort_name']).path.count().idxmax()

('spring/fundamentals', 'Staff')

In [45]:
# ds_df
ds_df.groupby('cohort_name')['path','user_id'].nunique()

,path,user_id
cohort_name,,
Bayes,532,23
Curie,417,21
Darden,419,29


In [52]:
# idxmax to get the most viewed module/lesson
ds_df.groupby(['module/lesson','cohort_name']).path.count().idxmax()

('classification/overview', 'Darden')

In [46]:
# fe_df
fe_df.groupby('cohort_name')['path','user_id'].nunique()

,path,user_id
cohort_name,,
Apollo,3,1


In [53]:
# idxmax to get the most viewed module/lesson
fe_df.groupby(['module/lesson','cohort_name']).path.count().idxmax()

('content/html-css', 'Apollo')

2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [59]:
# making a list for data science cohorts
ds_cohort_list = list(set(ds_df.cohort_name.to_list()))

In [60]:
# running through the data science cohorts
for cohort in ds_cohort_list:
    temp_df = df[df.cohort_name == cohort]
    print(cohort)
    print('------------------------------------------------------------')
    print(temp_df.groupby(['module/lesson']).count_helper.agg(['count']).sort_values(by = 'count',ascending = False).head(10))
    print('------------------------------------------------------------')

Darden
------------------------------------------------------------
                                                count
module/lesson                                        
classification/overview                          1109
classification/scale_features_or_not.svg          943
sql/mysql-overview                                774
search/search_index.json                          664
anomaly-detection/AnomalyDetectionCartoon.jpeg    612
anomaly-detection/overview                        592
1-fundamentals/AI-ML-DL-timeline.jpg              470
1-fundamentals/modern-data-scientist.jpg          470
1-fundamentals/1.1-intro-to-data-science          460
stats/compare-means                               423
------------------------------------------------------------
Bayes
------------------------------------------------------------
                                                   count
module/lesson                                           
1-fundamentals/modern-data-scientist.jpg  

3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [61]:
df.head()

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,count_helper,program_name,program_subdomain,slack,module/lesson
datetime,,,,,,,,,,,,,
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,NaN
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,java-ii/object-oriented-programming
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,slides/object_oriented_programming
2018-01-26 10:40:15,javascript-i/functions,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,javascript-i/functions
2018-01-26 11:26:13,java-i,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,1,PHP Full Stack Web Development,php,#hampton,NaN


4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP 
addresses?


5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?


In [62]:
# let's make a temp_df of 2019 to look at
temp_df = df.loc['2019']
temp_df.head()

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,count_helper,program_name,program_subdomain,slack,module/lesson
datetime,,,,,,,,,,,,,
2019-01-02 09:39:38,search/search_index.json,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,search/search_index.json
2019-01-02 09:39:53,javascript-ii/ajax-api,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,javascript-ii/ajax-api
2019-01-02 09:40:15,java-iii,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,NaN
2019-01-02 09:40:18,java-iii/servlets,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,java-iii/servlets
2019-01-02 09:40:40,java-iii/jdbc,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,java-iii/jdbc


In [65]:
# now let's get the web dev and ds programs java == 2; ds == 3
temp_df = temp_df[(temp_df.program_id == 2) | (temp_df.program_id == 3)]

In [66]:
temp_df.head() # check_yo_head

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,count_helper,program_name,program_subdomain,slack,module/lesson
datetime,,,,,,,,,,,,,
2019-01-02 09:39:38,search/search_index.json,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,search/search_index.json
2019-01-02 09:39:53,javascript-ii/ajax-api,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,javascript-ii/ajax-api
2019-01-02 09:40:15,java-iii,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,NaN
2019-01-02 09:40:18,java-iii/servlets,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,java-iii/servlets
2019-01-02 09:40:40,java-iii/jdbc,99,22,199.59.33.126,Teddy,2018-01-08,2018-05-17,2.0,1,Java Full Stack Web Development,java,#teddy,java-iii/jdbc
